<a href="https://colab.research.google.com/github/JoaoAnjos-DataScience/JoaoAnjos-DataScience/blob/main/teste_jo%C3%A3o_grafos_melhores_compras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Carregando os dados

In [20]:
!pip install cairocffi
!pip install python-igraph
!pip install python-louvain

In [3]:
import pandas as pd
users_df = pd.read_csv("/content/base bx completa.csv", encoding="iso-8859-1")
users_df

,User_ID,City,Estate,Country,Age,ISBN,Book_Rating,USR,Book-Title,Book-Author,Year-Of-Publication,Publisher,ISB
0,276853,lugo,galiza,spain,17,8420457477,1,276853,Los Trapos Sucios - Manolito Gafotas,Lindo,0,European Schoolbooks Limited,8420457477
1,276929,collex,geneva,switzerland,35,2266104535,2,276929,Et Si C'Etait Vrai / If This Were Only True,Marc Levy,2001,Pocket,2266104535
2,276994,wien,wien,austria,25,3478387507,2,276994,Die 22 grÃ?Â¶Ã?Ã?ten Management- Fehler und...,James K. Van Fleet,2001,Moderne Industrie,3478387507
3,276994,wien,wien,austria,25,3596154766,2,276994,Silberfaden.,Ricarda Junge,2002,Fischer (Tb.),3596154766
4,278054,verona,veneto,italy,40,1842322753,2,278054,An Old Captivity,Nevil Shute,2000,House of Stratus,1842322753
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185157,276680,hopewell junction,new york,usa,55,1573221562,10,276680,Affinity,Sarah Waters,2000,Riverhead Books,1573221562
185158,276680,hopewell junction,new york,usa,55,1573222038,10,276680,Fingersmith,Sarah Waters,2002,Riverhead Hardcover,1573222038
185159,276681,chicago,illinois,usa,43,014006284X,10,276681,Portrait of the Artist as a Young Man,James Joyce,1982,Penguin Books,014006284X
185160,276681,chicago,illinois,usa,43,1931333246,10,276681,Chorus,Kurt Messick,2003,Dry Bones Press,1931333246


Verificando as comundades

In [4]:
import igraph as ig
import pandas as pd

# Construindo o grafo
tuples = [tuple(x) for x in users_df[['User_ID', 'ISBN']].values]
g = ig.Graph.TupleList(tuples, directed=False)
# Mostrando a quantidade de nós e arestas
print("Número de nós:", g.vcount())
print("Número de arestas:", g.ecount())
# Detecção de comunidades com Louvain
louvain = g.community_multilevel()
# Mostrando a quantidade de comunidades detectadas
print("Número de comunidades detectadas:", len(louvain))
# Calculando centralidades
degree = g.degree()
betweenness = g.betweenness()
closeness = g.closeness()
eigenvector = g.eigenvector_centrality()
# Criando um DataFrame para armazenar as centralidades
centralidades = pd.DataFrame({
    'Centralidade de Grau': degree,
    'Centralidade de Intermediação': betweenness,
    'Centralidade de Proximidade': closeness,
    'Centralidade de Autovalor': eigenvector
})
# Convertendo os índices em coluna para melhor visualização
centralidades.reset_index(inplace=True)
centralidades.rename(columns={'index': 'Nó'}, inplace=True)
# Exibindo as centralidades dos primeiros 5 nós
print(centralidades.head())


Número de nós: 105370
Número de arestas: 185162
Número de comunidades detectadas: 5381
   Nó  Centralidade de Grau  Centralidade de Intermediação  \
0   0                     2                   9.315900e+04   
1   1                     4                   7.421300e+05   
2   2                    53                   3.450613e+06   
3   3                    32                   8.804820e+06   
4   4                    18                   1.449159e+06   

   Centralidade de Proximidade  Centralidade de Autovalor  
0                     0.130727               4.758043e-08  
1                     0.150386               2.379813e-06  
2                     0.188849               6.829030e-04  
3                     0.210422               2.407976e-03  
4                     0.181618               1.458640e-04  


In [19]:
import igraph as ig
import pandas as pd

user_id = 276680  # Substitua com o ID real do usuário
# Encontrando o vértice correspondente ao usuário no grafo
user_vertex_index = g.vs.find(name=user_id).index
# Encontrando os vizinhos (outros usuários com interesses de leitura similares)
neighbors = g.neighbors(user_vertex_index, mode='out')
# Coletando todos os livros que esses vizinhos leram
neighbor_books_indices = set()
for n in neighbors:
    neighbor_books_indices.update(g.neighbors(n, mode='out'))
# Convertendo índices para ISBNs
neighbor_books_isbns = {g.vs[book_idx]['name'] for book_idx in neighbor_books_indices}
# Livros que o usuário já leu (convertendo índices para ISBNs)
user_books_indices = set(g.neighbors(user_vertex_index, mode='out'))
user_books_isbns = {g.vs[book_idx]['name'] for book_idx in user_books_indices}
# Recomendando livros que os vizinhos leram, mas o usuário ainda não
recommended_books_isbns = list(neighbor_books_isbns - user_books_isbns)
# Selecionar os livros mais populares entre as recomendações
book_popularity = {book: g.degree(g.vs.find(name=book).index) for book in recommended_books_isbns}
recommended_books_sorted = sorted(book_popularity, key=book_popularity.get, reverse=True)
# Mostrando os top N recomendações
top_n = 5
print("Top", top_n, "recomendações de novos livros para o usuário", user_id, ":\n", recommended_books_sorted[:top_n])


Top 5 recomendações de novos livros para o usuário 276680 :
 [98391, 35859, 16795, 230522, 234623]
